In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 3.1 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np


In [3]:
print(tf.__version__)

2.6.0


In [4]:
fashion_mist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
# we will scale down this images in scale of 0 to 1 so that we can train our model
train_images = train_images/255.0 # these are the pixels that are present in every image
test_images = test_images/255.0

for CNN we have to reshape this data

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),    # 1st layer of convulation with filter= between 32 to 128 
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), # kernel ie filter of 3* or 5*5
        activation='relu',  
        input_shape=(28,28,1)   # shape of input
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16), # similarly second layer
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),     
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),  # flatten vector 
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')   # dense fully connected layer
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),  # kind of back propogation to optimize a loss using adam optimizer
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
# max trials will run 4 times to give different values of like filters all hyperparameters 
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=4,directory='output',project_name='Mnist Fashion')


In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1) # so this is like fitting x an y and epoch 3 will give best parameters till 3 pochs

Trial 4 Complete [00h 00m 41s]
val_accuracy: 0.8744999766349792

Best val_accuracy So Far: 0.9048333168029785
Total elapsed time: 00h 04m 12s
INFO:tensorflow:Oracle triggered exit


In [18]:
model= tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        27712     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1982528   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 2,012,138
Trainable params: 2,012,138
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images,train_labels,epochs=8,validation_split=0.1,initial_epoch=3,verbose=1)

Epoch 4/8
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0488 - accuracy: 0.9821 - val_loss: 0.4102 - val_accuracy: 0.9148
Epoch 5/8
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0371 - accuracy: 0.9867 - val_loss: 0.4280 - val_accuracy: 0.9132
Epoch 6/8
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0321 - accuracy: 0.9888 - val_loss: 0.4800 - val_accuracy: 0.9145
Epoch 7/8
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0299 - accuracy: 0.9896 - val_loss: 0.5516 - val_accuracy: 0.9065
Epoch 8/8
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0245 - accuracy: 0.9915 - val_loss: 0.5933 - val_accuracy: 0.9090
